In [ ]:
!pip install chromadb langchain-community langchain-text-splitters sentence-transformers tqdm





     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 124.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 115.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/

In [ ]:
import os
from tqdm import tqdm
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_community.document_loaders import TextLoader


DATA_DIR = "/content/"
DB_DIR = "/content/vectordb"


documents = []

print("Loading textbook files...")

for lang_folder in ["EN", "ZH"]:
    folder_path = os.path.join(DATA_DIR, lang_folder)

    for file in os.listdir(folder_path):
        if file.endswith(".txt"):
            file_path = os.path.join(folder_path, file)

            print("Loading:", file_path)

            loader = TextLoader(file_path, encoding="utf-8")
            docs = loader.load()

            for d in docs:
                d.metadata["language"] = "en" if lang_folder == "EN" else "zh"
                d.metadata["source"] = file

            documents.extend(docs)


print("Total raw documents loaded:", len(documents))


print("Splitting into chunks...")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=100
)

chunks = text_splitter.split_documents(documents)

print("Total chunks created:", len(chunks))


print("Loading embedding model...")

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)


print("Creating Chroma vector database...")

vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    persist_directory=DB_DIR
)

vectordb.persist()

print("✅ Vector database successfully created and saved!")


Loading textbook files...
Loading: /content/EN/Obstentrics_Williams.txt
Loading: /content/EN/Immunology_Janeway.txt
Loading: /content/EN/Histology_Ross.txt
Loading: /content/EN/Anatomy_Gray.txt
Loading: /content/EN/Pathology_Robbins.txt
Loading: /content/EN/InternalMed_Harrison.txt
Loading: /content/EN/Pediatrics_Nelson.txt
Loading: /content/EN/Psichiatry_DSM-5.txt
Loading: /content/EN/First_Aid_Step1.txt
Loading: /content/EN/Gynecology_Novak.txt
Loading: /content/EN/Cell_Biology_Alberts.txt
Loading: /content/EN/Pathoma_Husain.txt
Loading: /content/EN/Pharmacology_Katzung.txt
Loading: /content/EN/Surgery_Schwartz.txt
Loading: /content/EN/Biochemistry_Lippincott.txt
Loading: /content/EN/First_Aid_Step2.txt
Loading: /content/EN/Neurology_Adams.txt
Loading: /content/EN/Physiology_Levy.txt
Loading: /content/ZH/all_books.txt
Total raw documents loaded: 19
Splitting into chunks...
Total chunks created: 285189
Loading embedding model...


/tmp/ipython-input-2791429773.py:55: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Creating Chroma vector database...


KeyboardInterrupt: 

In [ ]:
import os
from tqdm import tqdm
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader


DATA_DIR = "/content/"
DB_DIR = "/content/vectordb"

documents = []

print("Loading textbook files...")

for lang_folder in ["EN", "ZH"]:
    folder_path = os.path.join(DATA_DIR, lang_folder)

    for file in os.listdir(folder_path):
        if file.endswith(".txt"):
            file_path = os.path.join(folder_path, file)

            print("Loading:", file_path)

            loader = TextLoader(file_path, encoding="utf-8")
            docs = loader.load()

            for d in docs:
                d.metadata["language"] = "en" if lang_folder == "EN" else "zh"
                d.metadata["source"] = file

            documents.extend(docs)

print("Total raw documents loaded:", len(documents))


print("Splitting into chunks...")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=100
)

chunks = text_splitter.split_documents(documents)

print("Total chunks created:", len(chunks))


print("Loading embedding model on GPU...")

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={"device": "cuda"},   # ✅ GPU
    encode_kwargs={"batch_size": 64}   # ✅ Faster on GPU
)

print("Creating Chroma vector database...")

vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    persist_directory=DB_DIR
)

vectordb.persist()

print("✅ Vector database successfully created and saved using GPU!")


Loading textbook files...
Loading: /content/EN/Obstentrics_Williams.txt
Loading: /content/EN/Immunology_Janeway.txt
Loading: /content/EN/Histology_Ross.txt
Loading: /content/EN/Anatomy_Gray.txt
Loading: /content/EN/Pathology_Robbins.txt
Loading: /content/EN/InternalMed_Harrison.txt
Loading: /content/EN/Pediatrics_Nelson.txt
Loading: /content/EN/Psichiatry_DSM-5.txt
Loading: /content/EN/First_Aid_Step1.txt
Loading: /content/EN/Gynecology_Novak.txt
Loading: /content/EN/Cell_Biology_Alberts.txt
Loading: /content/EN/Pathoma_Husain.txt
Loading: /content/EN/Pharmacology_Katzung.txt
Loading: /content/EN/Surgery_Schwartz.txt
Loading: /content/EN/Biochemistry_Lippincott.txt
Loading: /content/EN/First_Aid_Step2.txt
Loading: /content/EN/Neurology_Adams.txt
Loading: /content/EN/Physiology_Levy.txt
Loading: /content/ZH/all_books.txt
Total raw documents loaded: 19
Splitting into chunks...
Total chunks created: 285189
Loading embedding model on GPU...


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Creating Chroma vector database...
✅ Vector database successfully created and saved using GPU!


/tmp/ipython-input-1190695495.py:65: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [ ]:
!zip -r vectordb.zip /content/vectordb


  adding: content/vectordb/ (stored 0%)
  adding: content/vectordb/chroma.sqlite3 (deflated 51%)
  adding: content/vectordb/b107d99e-e3ff-4f3c-9adb-bf7ecf48ca37/ (stored 0%)
  adding: content/vectordb/b107d99e-e3ff-4f3c-9adb-bf7ecf48ca37/index_metadata.pickle (deflated 43%)
  adding: content/vectordb/b107d99e-e3ff-4f3c-9adb-bf7ecf48ca37/length.bin (deflated 81%)
  adding: content/vectordb/b107d99e-e3ff-4f3c-9adb-bf7ecf48ca37/header.bin (deflated 55%)
  adding: content/vectordb/b107d99e-e3ff-4f3c-9adb-bf7ecf48ca37/link_lists.bin (deflated 73%)
  adding: content/vectordb/b107d99e-e3ff-4f3c-9adb-bf7ecf48ca37/data_level0.bin (deflated 11%)


In [ ]:
from google.colab import files
files.download("/content/vectordb.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
pip install langchain